In [238]:
#####ノイズあり対応トピックモデル#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import gensim
from scipy.special import gammaln
from scipy.misc import factorial
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

####データの発生####
##データの設定
#文書の設定
k = 15   #トピック数
d = 5000   #文書数
v  = 1000   #語彙数
w = np.random.poisson(np.random.gamma(65, 1/0.5, d), d)   #1文書あたりの単語数
a1 = 150   #トピックに関係のあるタグ数
a2 = 100   #トピックに関係のないタグ数
a = a1 + a2
x = np.random.poisson(40, d)
x[x < 2] = 3
f1 = np.sum(w)
f2 = np.sum(x)

#IDの設定
w_id = np.repeat(range(d), w)
a_id = np.repeat(range(d), x)

In [239]:
##パラメータの設定
#ディレクリ事前分布のパラメータを設定
alpha0 = np.repeat(0.15, k)   #文書のディレクリ事前分布のパラメータ
alpha1 = np.repeat(0.05, v)   #単語のディレクリ事前分布のパラメータ
alpha2 = np.hstack((np.repeat(0.2, a1), np.repeat(0.001, a2)))   #トピックに関係のあるタグのディレクリ事前分布のパラメータ
alpha3 = np.hstack((np.repeat(0.001, a1), np.repeat(1.25, a2)))   #トピックに関係のないタグのディリクレ事前分布のパラメータ
beta0 = betat0 = np.random.beta(5.0, 2.0, d)

In [240]:
##すべての単語が出現するまで生成を繰り返す
rp = 0
while True:
    rp = rp + 1
    print(rp)
    
    #ディリクレ事前分布からパラメータを生成
    theta = thetat = np.random.dirichlet(alpha0, d)   #文書ごとのトピック分布
    phi = np.random.dirichlet(alpha1, k)   #トピックごとの単語分布
    omega = omegat = np.random.dirichlet(alpha2, k)   #トピックに関係のある補助情報分布
    gamma = gammat = np.random.dirichlet(alpha3, 1)   #トピックに関係のない補助情報分布
    omega0 = np.vstack((omega, gamma))   #補助情報分布の結合
    
    #出現確率が低い単語を入れ替える
    index_v = np.array(range(v))[np.max(phi, axis=0) <= (k*k*3)/f1]
    for j in range(index_v.shape[0]):
        phi[np.argmax(np.random.multinomial(1, np.repeat(1/k, k), 1)), index_v[j]] = (k*k*3)/f1
    phit = phi = phi / np.sum(phi, axis=1).reshape(k, 1)
    
    #データの格納用配列
    WX = np.zeros((d, v))
    AX = np.zeros((d, a))
    aux_data = np.zeros((f2, a))
    word_list = [i for i in range(d)]
    aux_list = [i for i in range(d)]
    Z0 = np.repeat(0, f2)
    Z1_list = [i for i in range(d)]
    Z2_list = [i for i in range(d)]
    vec = np.arange(0, k)

    ##文書ごとにトピックと単語を生成する
    for i in range(d):
        
        #文書のトピックを生成
        z1 = np.random.multinomial(1, theta[i, :], w[i])
        z1_vec = np.dot(z1, vec)

        #文書のトピック分布から単語を生成
        word = np.zeros((w[i], v))
        for j in range(w[i]):
            word[j, :] = multinomial(1, phi[z1_vec[j], :], 1)
        WX[i, :] = np.sum(word, axis=0)


        #ベルヌーイ分布からトピックに関係があるかどうかを生成
        index = np.arange(f2)[a_id==i]
        Z0[index] = np.random.binomial(1, beta0[i], x[i])

        #文書のトピック分布から補助情報トピックを生成
        z2_aux = np.random.multinomial(1, theta[i, :], x[i])
        z2 = np.concatenate((z2_aux * Z0[index].reshape((x[i], 1)),1-Z0[index].reshape((x[i], 1))), axis=1)
        z2_vec = np.dot(z2, np.hstack((vec, k)))

        #生成したトピックの単語分布に従い単語を生成
        aux = np.zeros((x[i], a))
        for j in range(x[i]):
            aux[j, :] = np.random.multinomial(1, omega0[z2_vec[j], :], 1)
        AX[i, :] = np.sum(aux, axis=0)

        #文書トピックおよび補助情報トピックを格納
        Z1_list[i] = z1_vec
        Z2_list[i] = z2_vec
        word_list[i] = np.dot(word, np.arange(v))
        aux_list[i] = np.dot(aux, np.arange(a))
        aux_data[a_id==i, :] = aux

    print(np.min(np.sum(WX, axis=0)))
    if (np.min(np.sum(WX, axis=0)) > 0.0) & (np.min(np.sum(AX, axis=0)) > 0.0):
        break

1
34.0


In [241]:
#リストをベクトル変換
wd = np.zeros(sum(w), dtype='int')
ad = np.zeros(sum(x), dtype='int')
start1 = start2 = 0

for i in range(d):
    wd[start1:start1+w[i]] = word_list[i]
    ad[start2:start2+x[i]] = aux_list[i]
    start1 += w[i] 
    start2 += x[i]

In [245]:
##インデックスを作成
doc1_list = [i for i in range(d)]
doc2_list = [i for i in range(d)]
doc1_vec = [i for i in range(d)]
doc2_vec = [i for i in range(d)]
word_list = [i for i in range(v)]
word_vec = [i for i in range(v)]
aux_list = [i for i in range(a)]
aux_vec = [i for i in range(a)]
index_doc = np.array(range(f1))
index_aux = np.array(range(f2))

for i in range(d):
    doc1_list[i] = index_doc[w_id==i]
    doc1_vec[i] = np.repeat(1, doc1_list[i].shape)
    doc2_list[i] = index_aux[a_id==i]
    doc2_vec[i] = np.repeat(1, doc2_list[i].shape)
for i in range(v):
    word_list[i] = index_doc[wd==i]
    word_vec[i] = np.repeat(1, word_list[i].shape)
for i in range(a):
    aux_list[i] = index_aux[ad==i]
    aux_vec[i] = np.repeat(1, aux_list[i].shape)

In [260]:
####ベイズ推定のための設定####
##単語ごとに尤度と負担率を計算する関数
def burden_fr(f, theta, phi, wd, w_id, k):
    #尤度を計算
    Bur = theta[w_id, :]  * phi.T[wd, ]   #トピック尤度

    #負担率を計算
    Br = Bur / np.sum(Bur, axis=1).reshape((f, 1))   #負担率
    r = np.sum(Br, axis=0) / np.sum(Br)
    return Br, Bur, r


##アルゴリズムの設定
R = 3000   #サンプリング回数
keep = 2   #2回に1回の割合でサンプリング結果を格納
disp = 10
iter = 0
burnin = 500

##事前分布の設定
alpha01 = 0.1
alpha02 = 0.1
beta01 = 0.1
beta02 = 0.1
gamma01 = 0.5
gamma02 = 0.5

##パラメータの初期値
#tf-idfで初期値を設定
tf = AX / np.sum(AX, axis=1).reshape((d, 1))
idf1 = np.log(AX.shape[0] / np.sum(AX > 0, axis=0))
idf2 = np.log(AX.shape[0] / np.sum(AX==0, axis=0))

##パラメータの真値
theta = thetat 
phi = phit
omega = omegat
gamma = gammat.reshape((a))
r = betat0

##パラメータの初期値
theta = np.random.dirichlet(np.repeat(1, k), d)   #文書トピックのパラメータの初期値
phi = np.random.dirichlet(np.repeat(2.0, v), k)   #単語分布のパラメータの初期値
omega = np.random.dirichlet(np.repeat(10, a), k)   #内容に関係のあるタグ分布のパラメータの初期値
gamma = np.random.dirichlet(np.repeat(10, a), 1).reshape((a))   #内容に関係のないタグ分布のパラメータの初期値
r = np.repeat(0.5, d)   #内容に関係があるかどうかの混合率

##パラメータの格納用配列
THETA = np.zeros((d, k, R//keep), dtype='float')
PHI = np.zeros((k, v, R//keep), dtype='float')
OMEGA = np.zeros((k, a, R//keep), dtype='float')
GAMMA = np.zeros((R//keep, a), dtype='float')
LAMBDA = np.zeros((R//keep, d), dtype='float')
S_SEG = np.zeros(f2, dtype='int')
W_SEG = np.zeros((f1, k), dtype='int')
A_SEG = np.zeros((f2, k), dtype='int')

##対数尤度の基準値
LLst1 = np.sum(np.log((np.sum(WX, axis=0) / f1)[wd]))
LLst2 = np.sum(np.log((np.sum(AX, axis=0) / f2)[ad]))
LLst = LLst1 + LLst2
print(np.round(np.array((LLst, LLst1, LLst2)), 1))

[-5221394.4 -4165328.2 -1056066.2]


In [261]:
####ノイズあり対応トピックモデルをベイズ推定####
for rp in range(R): 

    ##単語トピックをサンプリング
    #単語ごとにトピックの出現確率を計算
    word_par = burden_fr(f1, theta, phi, wd, w_id, k)
    Br = word_par[0]   #トピック出現確率

    #多項分布よりトピックをサンプリング
    Zi1 = np.array([np.random.multinomial(1, Br[i, :], 1) for i in range(f1)]).reshape((f1, k))
    Zi1_T = Zi1.T

    ##単語トピックのパラメータを更新
    #ディリクレ分布からthetaをサンプリング
    wsum = np.zeros((d, k))
    for i in range(d):
        wsum[i, :] = np.dot(Zi1_T[:, doc1_list[i]], doc1_vec[i]) + alpha01   #ディリクレ分布のパラメータ
        theta[i, :] = np.random.dirichlet(wsum[i, :], 1)   

    #ディリクレ分布からphiを更新
    vsum = np.zeros((k, v))
    for j in range(v):
        vsum[:, j] = np.dot(Zi1_T[:, word_list[j]], word_vec[j]) + beta01   #ディリクレ分布のパラメータ
    phi = np.array([np.random.dirichlet(vsum[i, :], 1) for i in range(k)]).reshape((k, v))


    ##補助情報がトピックに関係があるかどうかをサンプリング
    #ベルヌーイ分布の確率を設定
    r_vec = r[a_id]   #混合率のベクトル
    theta_aux = (wsum - alpha01) / w.reshape(d, 1)   #観測トピックからタグのトピック分布を決定  
    tau01 = r_vec * np.sum(theta_aux[a_id, :] * omega.T[ad, :], axis=1)   #補助変数トピックの尤度
    tau02 = (1-r_vec) * gamma[ad]   #ノイズ補助変数の尤度
    tau1 = tau01 / (tau01 + tau02)   #潜在変数の割当確率

    #ベルヌーイ分布から潜在変数をサンプリング
    S = numpy.random.binomial(1, tau1, f2)
    index_s = np.array(range(f2))[S==1]   #トピックに関係のあるタグのインデックス
    
    #ベータ分布より混合率の期待値をサンプリング
    #ベータ分布のパラメータ
    beta = np.zeros((d, 2))
    for i in range(d):
        n = np.sum(S[doc2_list[i]])
        beta[i, 0] = n + gamma01
        beta[i, 1] = doc2_list[i].shape[0] - n + gamma02

    #ベータ分布より混合率をサンプリング
    r = numpy.random.beta(beta[:, 0], beta[:, 1], d)

    
    ##補助情報トピックをサンプリング
    #タグごとにトピックの出現確率を計算
    n = np.sum(S) 
    aux_par = theta_aux[a_id, :] * omega.T[ad, :]   #タグのトピックの尤度
    aux_rate = (aux_par / np.sum(aux_par, axis=1).reshape(f2, 1))[index_s, :]  #トピックの割当確率

    #多項分布分布からトピックをサンプリング
    Zi2 = np.zeros((f2, k))
    Zi2[index_s, :] = np.array([np.random.multinomial(1, aux_rate[i, :], 1) for i in range(n)]).reshape((n, k))
    Zi2_T = Zi2.T
    
    ##補助情報トピックのパラメータをサンプリング
    #ディリクレ分布からomegaをサンプリング
    af = np.zeros((k, a))
    for j in range(a):
        af[:, j] = np.dot(Zi2_T[:, aux_list[j]], aux_vec[j]) + gamma01   #ディリクレ分布のパラメータ
    omega = np.array([np.random.dirichlet(af[i, :], 1) for i in range(k)]).reshape((k, a))

    #内容に関係のない補助情報分布omegaをサンプリング
    nf = np.sum(aux_data * (1-S).reshape(f2, 1), axis=0) + gamma02   #ディリクレ分布分布のパラメータ
    gamma = numpy.random.dirichlet(nf, 1).reshape(-1)


    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        mkeep = 0
        THETA[:, :, mkeep] = theta
        PHI[:, :, mkeep] = phi
        OMEGA[:, :, mkeep] = omega
        GAMMA[mkeep, :] = gamma
        LAMBDA[mkeep, :] = r

    #トピック割当はバーンイン期間を超えたら格納
    if rp%keep==0 & rp >= burnin:
        W_SEG = W_SEG + Zi1
        A_SEG = A_SEG + Zi2
        S_SEG = S_SEG + S 
        
    if rp%disp==0:
        #対数尤度の更新
        LL1 = np.sum(np.log(np.sum(word_par[1], axis=1)))
        LL2 = np.sum(np.log(gamma[ad][S==0])) + np.sum(np.log(np.sum(aux_par[index_s, :], axis=1)))
        LL = LL1 + LL2

        #サンプリング結果を確認
        print(rp)
        print(np.round(np.array((LL, LLst)), 1))
        print(np.round(np.vstack((gamma[(a1-6):(a1+6)], (gammat.reshape(-1))[(a1-6):(a1+6)])), 3))

0
[-5588727.9 -5221394.4]
[[0.008 0.006 0.002 0.003 0.004 0.004 0.001 0.005 0.    0.001 0.    0.008]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
10
[-5067988.2 -5221394.4]
[[0.008 0.007 0.001 0.003 0.002 0.003 0.001 0.006 0.    0.001 0.    0.007]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
20
[-4831531.1 -5221394.4]
[[0.007 0.006 0.001 0.003 0.002 0.004 0.    0.007 0.    0.001 0.    0.007]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
30
[-4727875.6 -5221394.4]
[[0.007 0.006 0.001 0.002 0.002 0.004 0.001 0.008 0.    0.001 0.    0.009]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
40
[-4666356.1 -5221394.4]
[[0.004 0.005 0.001 0.001 0.001 0.004 0.001 0.014 0.001 0.001 0.    0.014]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
50
[-4619306.8 -5221394.4]
[[0.003 0.003 0.    0.001 0.001 0.002 0.002 0.018 0.001 0.002 0.001 0.018]
 [0.    0

460
[-4534182.9 -5221394.4]
[[0.001 0.002 0.    0.    0.    0.    0.002 0.024 0.001 0.002 0.001 0.023]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
470
[-4533377.1 -5221394.4]
[[0.001 0.001 0.    0.001 0.    0.001 0.003 0.024 0.001 0.001 0.001 0.025]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
480
[-4533737.8 -5221394.4]
[[0.001 0.    0.    0.    0.    0.    0.002 0.025 0.001 0.001 0.001 0.024]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
490
[-4531446.4 -5221394.4]
[[0.    0.    0.    0.    0.    0.001 0.003 0.024 0.001 0.001 0.001 0.023]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
500
[-4531984.7 -5221394.4]
[[0.001 0.    0.    0.    0.    0.001 0.003 0.024 0.001 0.001 0.001 0.024]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
510
[-4532941.9 -5221394.4]
[[0.001 0.001 0.    0.    0.001 0.001 0.003 0.026 0.001 0.001 0.001 0.024]
 [

920
[-4531708.1 -5221394.4]
[[0.001 0.002 0.    0.    0.    0.001 0.003 0.024 0.001 0.001 0.    0.023]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
930
[-4531423.7 -5221394.4]
[[0.001 0.    0.    0.    0.    0.001 0.003 0.025 0.001 0.002 0.001 0.023]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
940
[-4531940.  -5221394.4]
[[0.002 0.001 0.    0.    0.    0.001 0.003 0.026 0.001 0.001 0.001 0.025]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
950
[-4532878.3 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.001 0.002 0.025 0.001 0.002 0.    0.024]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
960
[-4532381.9 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.    0.002 0.025 0.001 0.001 0.001 0.023]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
970
[-4533004.3 -5221394.4]
[[0.001 0.002 0.    0.    0.    0.    0.003 0.025 0.001 0.001 0.001 0.025]
 [

1380
[-4532501.6 -5221394.4]
[[0.002 0.001 0.    0.    0.    0.001 0.002 0.023 0.001 0.001 0.001 0.025]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
1390
[-4533670.1 -5221394.4]
[[0.002 0.001 0.    0.    0.    0.    0.003 0.024 0.001 0.001 0.001 0.025]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
1400
[-4532710.  -5221394.4]
[[0.002 0.001 0.    0.    0.    0.001 0.002 0.024 0.001 0.001 0.001 0.024]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
1410
[-4532395.2 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.001 0.003 0.027 0.001 0.002 0.    0.025]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
1420
[-4532025.9 -5221394.4]
[[0.001 0.002 0.    0.    0.001 0.    0.002 0.025 0.001 0.002 0.    0.023]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
1430
[-4533730.4 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.001 0.003 0.024 0.001 0.002 0.    0.0

1840
[-4531508.9 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.    0.003 0.026 0.001 0.001 0.001 0.025]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
1850
[-4531543.1 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.001 0.003 0.025 0.001 0.002 0.    0.024]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
1860
[-4531131.6 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.001 0.003 0.026 0.001 0.002 0.    0.024]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
1870
[-4531884.4 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.001 0.003 0.025 0.001 0.002 0.001 0.024]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
1880
[-4533083.4 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.    0.003 0.026 0.001 0.001 0.001 0.024]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
1890
[-4533109.7 -5221394.4]
[[0.001 0.002 0.    0.    0.    0.001 0.003 0.026 0.001 0.002 0.    0.0

2300
[-4531407.3 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.    0.003 0.025 0.001 0.002 0.001 0.023]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
2310
[-4532634.6 -5221394.4]
[[0.002 0.001 0.    0.    0.    0.    0.002 0.027 0.001 0.002 0.    0.024]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
2320
[-4532654.3 -5221394.4]
[[0.001 0.    0.    0.    0.    0.001 0.003 0.024 0.001 0.002 0.001 0.023]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
2330
[-4533016.  -5221394.4]
[[0.002 0.001 0.    0.001 0.    0.    0.002 0.024 0.001 0.002 0.    0.023]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
2340
[-4532977.2 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.001 0.003 0.028 0.001 0.002 0.    0.025]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
2350
[-4532549.2 -5221394.4]
[[0.001 0.    0.    0.    0.    0.001 0.002 0.025 0.001 0.002 0.001 0.0

2760
[-4532184.7 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.001 0.002 0.025 0.001 0.001 0.001 0.023]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
2770
[-4533681.6 -5221394.4]
[[0.001 0.001 0.    0.    0.    0.    0.003 0.025 0.001 0.002 0.001 0.023]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
2780
[-4533479.8 -5221394.4]
[[0.002 0.002 0.    0.    0.    0.001 0.003 0.025 0.001 0.001 0.    0.026]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
2790
[-4532536.7 -5221394.4]
[[0.001 0.002 0.    0.    0.    0.    0.003 0.024 0.001 0.001 0.    0.026]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
2800
[-4532831.8 -5221394.4]
[[0.002 0.001 0.    0.    0.    0.001 0.003 0.026 0.001 0.001 0.    0.024]
 [0.    0.    0.    0.    0.    0.    0.003 0.027 0.002 0.002 0.001 0.026]]
2810
[-4532880.2 -5221394.4]
[[0.002 0.001 0.    0.    0.    0.001 0.003 0.024 0.002 0.002 0.001 0.0